In [36]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [37]:
import datetime
from langchain.output_parsers import JsonOutputToolsParser, PydanticToolsParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

In [67]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert researcher.
            Current Time: {time}
            1. {first_instruction}
            2. Reflect and critique your answer. Be severe to maximize improvement.
            3. Recommend search queries to research information and improve your answer
            (NOTE: You must provide the search queries at all costs)"""
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
actor_prompt_template = actor_prompt_template.partial(time=lambda:datetime.datetime.now().isoformat())

In [74]:
from schemas import AnswerQuestion


llm = ChatOpenAI()
parser_json = JsonOutputToolsParser(return_id=True)
parser_pydantic = PydanticToolsParser(tools=[AnswerQuestion])

In [75]:
first_responder_prompt_template = actor_prompt_template.partial(
    first_instruction="Provide a detailed ~250 word answer"
)

In [76]:
first_responder = first_responder_prompt_template | llm.bind_tools(
    tools=[AnswerQuestion], tool_choice="AnswerQuestion"
)

In [77]:
human_message = HumanMessage(
    content="""Write about AI-Powered SOC / autonomous SOC problem domain
    List startups that do that and raised capital"""
)

In [78]:
chain = (
    first_responder_prompt_template 
    | llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion")
    | parser_pydantic
)

In [79]:
res = chain.invoke(input={"messages": [human_message]})

In [80]:
for inst in res[0]:
    print(inst[0], end=": ")
    print(inst[1])
    print()
    print()
    print()

answer: AI-powered Security Operations Centers (SOC) or autonomous SOCs leverage artificial intelligence and machine learning algorithms to enhance threat detection, response, and remediation processes. These advanced systems can automatically analyze vast amounts of security data, identify patterns, and respond to security incidents in real-time, reducing manual intervention and response time. They enable proactive threat hunting, anomaly detection, and automated incident response, making cybersecurity operations more efficient and effective. Some notable startups in this domain include Darktrace, which has raised significant capital for its AI-driven cybersecurity solutions, and Securonix, a company that offers AI-based security analytics for threat detection and response. These startups have attracted investments due to the increasing demand for advanced cybersecurity solutions that can combat evolving cyber threats effectively.



reflection: missing='The answer could benefit from 